In [15]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [11]:
sqlContext = SQLContext(sc)

In [12]:
df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv',
                          format = 'com.databricks.spark.csv',
                          header = 'true',
                          inferSchema = 'true')

In [13]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [14]:
df.describe('avg_wind_speed_9am').show()

+-------+------------------+
|summary|avg_wind_speed_9am|
+-------+------------------+
|  count|              1092|
|   mean|  5.50828424225493|
| stddev| 4.552813465531715|
|    min|  0.69345139999974|
|    max|23.554978199999763|
+-------+------------------+



In [31]:
df.describe('rain_accumulation_9am').show()

+-------+---------------------+
|summary|rain_accumulation_9am|
+-------+---------------------+
|  count|                 1089|
|   mean|  0.20307895225211126|
| stddev|   1.5939521253574904|
|    min|                  0.0|
|    max|    24.01999999999907|
+-------+---------------------+



In [32]:
df.count() 

1095

In [33]:
df.stat.corr('relative_humidity_9am','relative_humidity_3pm')

0.883074190979391

In [20]:
df.describe('max_wind_speed_9am').show()

+-------+------------------+
|summary|max_wind_speed_9am|
+-------+------------------+
|  count|              1091|
|   mean| 7.019513529175272|
| stddev|  5.59820917078096|
|    min|1.1855782000000479|
|    max| 29.84077959999996|
+-------+------------------+



In [22]:
df.select('max_wind_direction_9am', 'max_wind_speed_9am').filter("max_wind_speed_9am > 29").show()

+----------------------+------------------+
|max_wind_direction_9am|max_wind_speed_9am|
+----------------------+------------------+
|      67.9999999999999| 29.84077959999996|
+----------------------+------------------+



In [28]:
import matplotlib.pyplot as plt
# Show histogram of the 'C1' column
bins, counts = df.select('air_temp_9am').rdd.flatMap(lambda x: x).histogram(50)

# This is a bit awkward but I believe this is the correct way to do it 
plt.hist(bins[:-1], bins=bins, weights=counts)

(array([  3.,   3.,   7.,   7.,  11.,   7.,  14.,  18.,  24.,  23.,  31.,
         20.,  15.,  28.,  18.,  24.,  33.,  34.,  38.,  43.,  43.,  40.,
         48.,  43.,  34.,  61.,  49.,  32.,  37.,  50.,  55.,  34.,  34.,
         22.,  29.,  13.,  22.,   9.,  10.,   9.,   8.,   1.,   3.,   2.,
          0.,   0.,   0.,   0.,   0.,   1.]),
 array([ 36.752  ,  37.99508,  39.23816,  40.48124,  41.72432,  42.9674 ,
         44.21048,  45.45356,  46.69664,  47.93972,  49.1828 ,  50.42588,
         51.66896,  52.91204,  54.15512,  55.3982 ,  56.64128,  57.88436,
         59.12744,  60.37052,  61.6136 ,  62.85668,  64.09976,  65.34284,
         66.58592,  67.829  ,  69.07208,  70.31516,  71.55824,  72.80132,
         74.0444 ,  75.28748,  76.53056,  77.77364,  79.01672,  80.2598 ,
         81.50288,  82.74596,  83.98904,  85.23212,  86.4752 ,  87.71828,
         88.96136,  90.20444,  91.44752,  92.6906 ,  93.93368,  95.17676,
         96.41984,  97.66292,  98.906  ]),
 <a list of 50 Patch ob

In [29]:
plt.show()

In [ ]:
print('57')